# Instalando o Pyspark no Google Colab

In [1]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# unzip the spark file to the current folder
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# install findspark and pyspark using pip
!pip install -q findspark
!pip install pyspark

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 39.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=fe5174b1501748ccc5e0e0e476de94e44e52c1f5cfded3fb44f2843cbc44fb62
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# Chamandos as libraries

In [2]:
import pandas as pd
import pyspark
import findspark
findspark.init()

In [9]:
# importing sparksession from pyspark.sql module
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *

In [10]:
spark = SparkSession.builder \
    .master('local') \
    .appName('DesafioStone') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
spark

# Input e análise de dados com Pandas

In [4]:
from google.colab import files
uploaded = files.upload()

Saving base_desafio_lucros.json to base_desafio_lucros.json


In [ ]:
pd_funcionarios = pd.read_json('base_desafio_lucros.json')
pd_funcionarios.head()

,matricula,nome,area,cargo,salario_bruto,data_de_admissao
0,5433534,Ivo Ferreira,Tecnologia,Pessoa Desenvolvedora,"$3,636.00",2004-05-28
1,5241166,Caetana da Magalhães,Tecnologia,Jovem Aprendiz,"$12,726.00",2009-09-06
2,3027629,Benjamim Cardoso,Contabilidade,Auxiliar Administrativo,"$3,636.00",2020-05-08
3,2556888,Diego da Campos,Relacionamento com o Cliente,Atendente,"$1,515.00",2016-09-10
4,2390033,Leonor Moura,Tecnologia,Jovem Aprendiz,"$2,424.00",2016-01-15


In [ ]:
pd_funcionarios.count()

matricula           1000
nome                1000
area                1000
cargo               1000
salario_bruto       1000
data_de_admissao    1000
dtype: int64

In [ ]:
pd_funcionarios['salario_bruto'].describe()

count          1000
unique           57
top       $7,272.00
freq             50
Name: salario_bruto, dtype: object

In [ ]:
#analisando a quantidade de funcionarios por cargo
pd_funcionarios['cargo'].value_counts()

Auxiliar Administrativo      118
Atendente de Almoxarifado     90
Jovem Aprendiz                86
Pessoa Desenvolvedora         83
Copeiro                       79
Diretor Tecnologia            65
Diretor Financeiro            63
Analista de Finanças          50
Atendente                     44
Economista Pleno              44
Economista Júnior             42
Líder de Relacionamento       38
Contador Júnior               37
Estagiário                    36
Auxiliar de Contabilidade     34
Auxiliar de Ouvidoria         33
Contador Pleno                31
Líder de Ouvidoria            27
Name: cargo, dtype: int64

In [ ]:
#analisando a quantidade de funcionarios por area
pd_funcionarios['area'].value_counts()

Diretoria                       206
Tecnologia                      169
Serviços Gerais                 169
Financeiro                      159
Contabilidade                   155
Relacionamento com o Cliente    142
Name: area, dtype: int64

In [ ]:
#validando se existem dados nulos
pd_funcionarios.apply(lambda x: sum(x.isnull()),axis=0)

matricula           0
nome                0
area                0
cargo               0
salario_bruto       0
data_de_admissao    0
dtype: int64

# Iniciando o desenvolvimento

In [11]:
df_funcionarios = spark.read.json('base_desafio_lucros.json') \
                  .select("nome"
                          ,"matricula"
                          ,upper(col("cargo")).alias("cargo")
                          ,upper(col("area")).alias("area")
                          ,regexp_replace(col("salario_bruto"), "[\$#,]", "").cast('float').alias("salario_bruto")
                          ,to_date("data_de_admissao").alias("data_de_admissao")
                          ,datediff(current_date(),col("data_de_admissao")).alias("dias_admissao")
                          )
                  
df_funcionarios.show(5, truncate=False)

+--------------------+---------+-----------------------+----------------------------+-------------+----------------+-------------+
|nome                |matricula|cargo                  |area                        |salario_bruto|data_de_admissao|dias_admissao|
+--------------------+---------+-----------------------+----------------------------+-------------+----------------+-------------+
|Ivo Ferreira        |5433534  |PESSOA DESENVOLVEDORA  |TECNOLOGIA                  |3636.0       |2004-05-28      |6548         |
|Caetana da Magalhães|5241166  |JOVEM APRENDIZ         |TECNOLOGIA                  |12726.0      |2009-09-06      |4621         |
|Benjamim Cardoso    |3027629  |AUXILIAR ADMINISTRATIVO|CONTABILIDADE               |3636.0       |2020-05-08      |724          |
|Diego da Campos     |2556888  |ATENDENTE              |RELACIONAMENTO COM O CLIENTE|1515.0       |2016-09-10      |2060         |
|Leonor Moura        |2390033  |JOVEM APRENDIZ         |TECNOLOGIA                 

In [ ]:
#Checando se existem matriculas duplicadas
df_funcionarios.select('matricula').dropDuplicates().count()

1000

In [ ]:
#analisando os tipos de cada variável
df_funcionarios.dtypes

[('nome', 'string'),
 ('matricula', 'bigint'),
 ('cargo', 'string'),
 ('area', 'string'),
 ('salario_bruto', 'float'),
 ('data_de_admissao', 'date'),
 ('dias_admissao', 'int')]

In [6]:
#definicao de variaveis

#valor disponivel para distribuicao
lucro_empresa_total = 5000000

#efetuado a divisão do valor total pelas 3 regras existentes
lucro_divisao_quadrante = lucro_empresa_total / 3 
print(lucro_divisao_quadrante)

1666666.6666666667


In [7]:
#valor salario minimo atualmente
salario_minimo_base = 1212.00

## Bonus por Area


```
Foi estabelecido um peso por área de atuação (Quanto maior o peso maior o valor recebido):

Peso 1: Diretoria;
Peso 2: Contabilidade, Financeiro, Tecnologia;
Peso 3: Serviços Gerais;
Peso 5: Relacionamento com o Cliente;


```
Diretoria                       206
Tecnologia                      169
Serviços Gerais                 169
Financeiro                      159
Contabilidade                   155
Relacionamento com o Cliente    142


In [21]:
##definindo os pesos de acordo com a area de cada funcionario
df_funcionarios_q1 = df_funcionarios \
                      .withColumn('peso_area_atuacao',
                                  when(df_funcionarios['area'] == 'DIRETORIA', 1)
                                  .when(df_funcionarios['area'].isin('CONTABILIDADE','FINANCEIRO','TECNOLOGIA'), 2)
                                  .when(df_funcionarios['area'] == 'SERVIÇOS GERAIS', 3)
                                  .when(df_funcionarios['area'] == 'RELACIONAMENTO COM O CLIENTE', 5)
                                  .otherwise(99)) ##em caso de uma area diferente, 
                                                  ##mantenho 99 para facilitar a evidencia do cenario nao esperado

In [ ]:
df_funcionarios_q1.filter(~df_funcionarios_q1['peso_area_atuacao'].isin(1,2,3,5)).show(5)

+----+---------+-----+----+-------------+-----------------+
|nome|matricula|cargo|area|salario_bruto|peso_area_atuacao|
+----+---------+-----+----+-------------+-----------------+
+----+---------+-----+----+-------------+-----------------+



In [ ]:
df_funcionarios_q1.show(10, truncate = False)

+----------------------------+-----------------------+----------------+---------+--------------------+-------------+----------------------------+-----------------+
|area                        |cargo                  |data_de_admissao|matricula|nome                |salario_bruto|area_upper                  |peso_area_atuacao|
+----------------------------+-----------------------+----------------+---------+--------------------+-------------+----------------------------+-----------------+
|Tecnologia                  |Pessoa Desenvolvedora  |2004-05-28      |5433534  |Ivo Ferreira        |$3,636.00    |TECNOLOGIA                  |2                |
|Tecnologia                  |Jovem Aprendiz         |2009-09-06      |5241166  |Caetana da Magalhães|$12,726.00   |TECNOLOGIA                  |2                |
|Contabilidade               |Auxiliar Administrativo|2020-05-08      |3027629  |Benjamim Cardoso    |$3,636.00    |CONTABILIDADE               |2                |
|Relacionamento 

In [22]:
df_funcionarios_q1_final = df_funcionarios_q1 \
                     .withColumn('bonus_quadrante1',
                                  when(df_funcionarios_q1['peso_area_atuacao'] == 1, salario_minimo_base)
                                  .when(df_funcionarios_q1['peso_area_atuacao'] == 2, salario_minimo_base * 1.25)
                                  .when(df_funcionarios_q1['peso_area_atuacao'] == 3, salario_minimo_base * 1.5)
                                  .when(df_funcionarios_q1['peso_area_atuacao'] == 5, salario_minimo_base * 2)
                                  .otherwise(0)) \
                      .select(df_funcionarios_q1['matricula'],
                              df_funcionarios_q1['nome'],
                              df_funcionarios_q1['area'],
                              df_funcionarios_q1['peso_area_atuacao'],
                              round(col('bonus_quadrante1'),2).alias('bonus_quadrante1')
                              )


In [ ]:
df_funcionarios_q1_final.show(20)

+---------+--------------------+--------------------+-----------------+----------------+
|matricula|                nome|          area_upper|peso_area_atuacao|bonus_quadrante1|
+---------+--------------------+--------------------+-----------------+----------------+
|  5433534|        Ivo Ferreira|          TECNOLOGIA|                2|          1515.0|
|  5241166|Caetana da Magalhães|          TECNOLOGIA|                2|          1515.0|
|  3027629|    Benjamim Cardoso|       CONTABILIDADE|                2|          1515.0|
|  2556888|     Diego da Campos|RELACIONAMENTO CO...|                5|          2424.0|
|  2390033|        Leonor Moura|          TECNOLOGIA|                2|          1515.0|
|  3489341|       Joel do Nunes|RELACIONAMENTO CO...|                5|          2424.0|
|  6406083| Lara-Francisca Lima|     SERVIÇOS GERAIS|                3|          1818.0|
|  4036167|   Vicente Domingues|       CONTABILIDADE|                2|          1515.0|
|  9204073|       Gus

In [32]:
df_soma_q1 = df_funcionarios_q1_final \
  .select(df_funcionarios_q1_final['bonus_quadrante1']) \
  .groupBy() \
  .sum() \

In [33]:
df_soma_q1.show()

+---------------------+
|sum(bonus_quadrante1)|
+---------------------+
|            1632867.0|
+---------------------+



In [ ]:
df_soma_q1.show()

+---------------------+
|sum(bonus_quadrante1)|
+---------------------+
|            1632867.0|
+---------------------+



In [ ]:
df_soma_q1.show()

+---------------------+
|sum(bonus_quadrante1)|
+---------------------+
|   1596809.9999999928|
+---------------------+



In [ ]:
print(1666666.6666666667 - 1632867)



33799.666666666744


In [ ]:
print(1666666.6666666667 - 1596809.9999999928)

69856.66666667396


In [ ]:
df_soma_q1 = df_funcionarios_q1_final.agg(sum("bonus_quadrante1")).collect()[0][0]

In [ ]:
print(df_soma_q1)#1.666.666.67

1548693.599999998


## Bonus Faixa Salarial


```
Foi estabelecido um peso por faixa salarial e uma exceção para estagiários(Quanto maior o peso menor o
valor recebido):
Peso 5: Acima de 8 salários mínimos;
Peso 3: Acima de 5 salários mínimos e menor que 8 salários mínimos;
Peso 2: Acima de 3 salários mínimos e menor que 5 salários mínimos;
Peso 1: Todos os estagiários e funcionários que ganham até 3 salários mínimos;
```

In [13]:
salario_minimo_base_mult_8 = salario_minimo_base * 8
salario_minimo_base_mult_5 = salario_minimo_base * 5
salario_minimo_base_mult_3 = salario_minimo_base * 3

In [ ]:
type(salario_minimo_base_mult_3)

float

In [ ]:
print(salario_minimo_base_mult_8)

9696.0


In [ ]:
print(salario_minimo_base_mult_5)

6060.0


In [ ]:
print(salario_minimo_base_mult_3)

3636.0


In [39]:
df_funcionarios_q2 = df_funcionarios \
                      .withColumn('peso_faixa_salarial',
                                  when(df_funcionarios['salario_bruto'] >= salario_minimo_base_mult_8, 5)
                                  .when((df_funcionarios['salario_bruto'] < salario_minimo_base_mult_8)
                                        & (df_funcionarios['salario_bruto'] >= salario_minimo_base_mult_5), 3)
                                  .when((df_funcionarios['salario_bruto'] < salario_minimo_base_mult_5)
                                        & (df_funcionarios['salario_bruto'] >= salario_minimo_base_mult_3), 2)     
                                  .when((df_funcionarios['salario_bruto'] < salario_minimo_base_mult_3)
                                        & (df_funcionarios['cargo']!='JOVEM APRENDIZ'), 1)                             
                                  .otherwise(99))
                                  ##Aqui surgiu uma dúvida para a regra de peso 1, como não foi citado, estou entendendo
                                  ##que o jovem aprendiz não terá direito a este bônus

In [ ]:
df_funcionarios_q2.select('cargo').distinct().show(truncate=False)

+-------------------------+
|cargo                    |
+-------------------------+
|LÍDER DE RELACIONAMENTO  |
|DIRETOR FINANCEIRO       |
|ESTAGIÁRIO               |
|ECONOMISTA JÚNIOR        |
|CONTADOR PLENO           |
|AUXILIAR DE CONTABILIDADE|
|AUXILIAR DE OUVIDORIA    |
|CONTADOR JÚNIOR          |
|ATENDENTE DE ALMOXARIFADO|
|ATENDENTE                |
|AUXILIAR ADMINISTRATIVO  |
|DIRETOR TECNOLOGIA       |
|JOVEM APRENDIZ           |
|COPEIRO                  |
|PESSOA DESENVOLVEDORA    |
|ECONOMISTA PLENO         |
|LÍDER DE OUVIDORIA       |
|ANALISTA DE FINANÇAS     |
+-------------------------+



In [ ]:
df_funcionarios_q2.show(20, truncate=False)

+--------------------+---------+-------------------------+----------------------------+-------------+-------------------+
|nome                |matricula|cargo                    |area                        |salario_bruto|peso_faixa_salarial|
+--------------------+---------+-------------------------+----------------------------+-------------+-------------------+
|Ivo Ferreira        |5433534  |PESSOA DESENVOLVEDORA    |TECNOLOGIA                  |3636.0       |2                  |
|Caetana da Magalhães|5241166  |JOVEM APRENDIZ           |TECNOLOGIA                  |12726.0      |5                  |
|Benjamim Cardoso    |3027629  |AUXILIAR ADMINISTRATIVO  |CONTABILIDADE               |3636.0       |2                  |
|Diego da Campos     |2556888  |ATENDENTE                |RELACIONAMENTO COM O CLIENTE|1515.0       |1                  |
|Leonor Moura        |2390033  |JOVEM APRENDIZ           |TECNOLOGIA                  |2424.0       |99                 |
|Joel do Nunes       |34

In [40]:
df_funcionarios_q2_final = df_funcionarios_q2 \
                     .withColumn('bonus_quadrante2',
                                  when(df_funcionarios_q2['peso_faixa_salarial'] == 5, salario_minimo_base)
                                  .when(df_funcionarios_q2['peso_faixa_salarial'] == 3, salario_minimo_base * 1.25)
                                  .when(df_funcionarios_q2['peso_faixa_salarial'] == 2, salario_minimo_base * 1.5)
                                  .when(df_funcionarios_q2['peso_faixa_salarial'] == 1, salario_minimo_base * 2)
                                  .otherwise(0)) \
                      .select(df_funcionarios_q2['matricula'],
                              df_funcionarios_q2['nome'],
                              df_funcionarios_q2['cargo'],
                              df_funcionarios_q2['peso_faixa_salarial'],
                              round(col('bonus_quadrante2'),2).alias('bonus_quadrante2')
                              )

In [ ]:
df_funcionarios_q2_final.show(5, truncate=False)

+---------+--------------------+-----------------------+-------------------+----------------+
|matricula|nome                |cargo                  |peso_faixa_salarial|bonus_quadrante2|
+---------+--------------------+-----------------------+-------------------+----------------+
|5433534  |Ivo Ferreira        |PESSOA DESENVOLVEDORA  |2                  |1818.0          |
|5241166  |Caetana da Magalhães|JOVEM APRENDIZ         |5                  |1212.0          |
|3027629  |Benjamim Cardoso    |AUXILIAR ADMINISTRATIVO|2                  |1818.0          |
|2556888  |Diego da Campos     |ATENDENTE              |1                  |2424.0          |
|2390033  |Leonor Moura        |JOVEM APRENDIZ         |99                 |0.0             |
+---------+--------------------+-----------------------+-------------------+----------------+
only showing top 5 rows



In [41]:
df_soma_q2 = df_funcionarios_q2_final \
  .select(df_funcionarios_q2_final['bonus_quadrante2']) \
  .groupBy() \
  .sum() \

In [42]:
df_soma_q2.show(5)

+---------------------+
|sum(bonus_quadrante2)|
+---------------------+
|            1664379.0|
+---------------------+



In [ ]:
df_soma_q2.show(5)

+---------------------+
|sum(bonus_quadrante2)|
+---------------------+
|            1664379.0|
+---------------------+



## Bonus tempo de admissao

```
Foi estabelecido um peso por tempo de admissão(Quanto maior o peso maior o valor recebido):
Peso 1: Até 1 ano de casa;
Peso 2: Mais de 1 ano e menos de 3 anos;
Peso 3: Acima de 3 anos e menos de 8 anos;
Peso 5: Mais de 8 anos
```

In [34]:
dias_admissao_1_ano = 365
dias_admissao_3_anos = 1095 
dias_admissao_8_anos = 2920

In [35]:
df_funcionarios_q3 = df_funcionarios \
                      .withColumn('peso_tempo_admissao',
                                  when(df_funcionarios['dias_admissao'] <= dias_admissao_1_ano, 1)
                                  .when((df_funcionarios['dias_admissao'] > dias_admissao_1_ano)
                                        & (df_funcionarios['dias_admissao'] <= dias_admissao_3_anos), 2)
                                  .when((df_funcionarios['dias_admissao'] > dias_admissao_3_anos)
                                        & (df_funcionarios['dias_admissao'] <= dias_admissao_8_anos), 3)                                 
                                  .otherwise(5)) ##acima de 8 anos

In [ ]:
df_funcionarios_q3.show(5)

+--------------------+---------+--------------------+--------------------+-------------+----------------+-------------+-------------------+
|                nome|matricula|               cargo|                area|salario_bruto|data_de_admissao|dias_admissao|peso_tempo_admissao|
+--------------------+---------+--------------------+--------------------+-------------+----------------+-------------+-------------------+
|        Ivo Ferreira|  5433534|PESSOA DESENVOLVE...|          TECNOLOGIA|       3636.0|      2004-05-28|         6546|                  4|
|Caetana da Magalhães|  5241166|      JOVEM APRENDIZ|          TECNOLOGIA|      12726.0|      2009-09-06|         4619|                  4|
|    Benjamim Cardoso|  3027629|AUXILIAR ADMINIST...|       CONTABILIDADE|       3636.0|      2020-05-08|          722|                  2|
|     Diego da Campos|  2556888|           ATENDENTE|RELACIONAMENTO CO...|       1515.0|      2016-09-10|         2058|                  3|
|        Leonor Mour

In [36]:
df_funcionarios_q3_final = df_funcionarios_q3 \
                     .withColumn('bonus_quadrante3',
                                  when(df_funcionarios_q3['peso_tempo_admissao'] == 1, salario_minimo_base * 0.75)
                                  .when(df_funcionarios_q3['peso_tempo_admissao'] == 2, salario_minimo_base * 0.95)
                                  .when(df_funcionarios_q3['peso_tempo_admissao'] == 3, salario_minimo_base * 1.15)
                                  .when(df_funcionarios_q3['peso_tempo_admissao'] == 5, salario_minimo_base * 1.55)
                                  .otherwise(0)) \
                      .select(df_funcionarios_q3['matricula'],
                              df_funcionarios_q3['nome'],
                              df_funcionarios_q3['data_de_admissao'],                              
                              df_funcionarios_q3['dias_admissao'],
                              df_funcionarios_q3['peso_tempo_admissao'],
                              round(col('bonus_quadrante3'),2).alias('bonus_quadrante3')
                              )

In [ ]:
df_funcionarios_q3_final.show(10)

+---------+--------------------+----------------+-------------+-------------------+----------------+
|matricula|                nome|data_de_admissao|dias_admissao|peso_tempo_admissao|bonus_quadrante3|
+---------+--------------------+----------------+-------------+-------------------+----------------+
|  5433534|        Ivo Ferreira|      2004-05-28|         6546|                  5|          2424.0|
|  5241166|Caetana da Magalhães|      2009-09-06|         4619|                  5|          2424.0|
|  3027629|    Benjamim Cardoso|      2020-05-08|          722|                  2|          1515.0|
|  2556888|     Diego da Campos|      2016-09-10|         2058|                  3|          1818.0|
|  2390033|        Leonor Moura|      2016-01-15|         2297|                  3|          1818.0|
|  3489341|       Joel do Nunes|      2011-03-07|         4072|                  5|          2424.0|
|  6406083| Lara-Francisca Lima|      2002-09-03|         7179|                  5|        

In [37]:
df_soma_q3 = df_funcionarios_q3_final \
  .select(df_funcionarios_q3_final['bonus_quadrante3']) \
  .groupBy() \
  .sum() \

In [38]:
df_soma_q3.show()

+---------------------+
|sum(bonus_quadrante3)|
+---------------------+
|   1644926.4000000171|
+---------------------+



In [ ]:
df_soma_q3.show()

+---------------------+
|sum(bonus_quadrante3)|
+---------------------+
|   1947926.4000000237|
+---------------------+



# JOIN com todos os Datraframes de Bonus

In [28]:
df_funcionarios_join_bonus = df_funcionarios \
                            .join(df_funcionarios_q1_final
                                  ,"matricula"
                                  ,"inner") \
                            .join(df_funcionarios_q2_final
                                  ,"matricula"
                                  ,"inner") \
                            .join(df_funcionarios_q3_final
                                  ,"matricula"
                                  ,"inner") \
                            .select(
                                df_funcionarios["matricula"]
                                ,df_funcionarios["nome"]
                                ,df_funcionarios["cargo"]
                                ,df_funcionarios["area"]
                                ,df_funcionarios["salario_bruto"]
                                ,df_funcionarios_q1_final["bonus_quadrante1"]
                                ,df_funcionarios_q2_final["bonus_quadrante2"]
                                ,df_funcionarios_q3_final["bonus_quadrante3"]
                            )

In [ ]:
df_funcionarios_join_bonus.show(10)

+---------+--------------------+--------------------+--------------------+-------------+----------------+----------------+----------------+
|matricula|                nome|               cargo|                area|salario_bruto|bonus_quadrante1|bonus_quadrante2|bonus_quadrante3|
+---------+--------------------+--------------------+--------------------+-------------+----------------+----------------+----------------+
|  5433534|        Ivo Ferreira|PESSOA DESENVOLVE...|          TECNOLOGIA|       3636.0|          1515.0|          1818.0|          2424.0|
|  5241166|Caetana da Magalhães|      JOVEM APRENDIZ|          TECNOLOGIA|      12726.0|          1515.0|          1212.0|          2424.0|
|  3027629|    Benjamim Cardoso|AUXILIAR ADMINIST...|       CONTABILIDADE|       3636.0|          1515.0|          1818.0|          1515.0|
|  2556888|     Diego da Campos|           ATENDENTE|RELACIONAMENTO CO...|       1515.0|          2424.0|          2424.0|          1818.0|
|  2390033|        L

In [ ]:
df_funcionarios_join_bonus.show(1)

+---------+------------+--------------------+----------+-------------+----------------+-------------+------------+----------+-----------------+----------------+------------+--------------------+-------------------+----------------+------------+----------------+-------------+-------------------+----------------+
|matricula|        nome|               cargo|      area|salario_bruto|data_de_admissao|dias_admissao|        nome|      area|peso_area_atuacao|bonus_quadrante1|        nome|               cargo|peso_faixa_salarial|bonus_quadrante2|        nome|data_de_admissao|dias_admissao|peso_tempo_admissao|bonus_quadrante3|
+---------+------------+--------------------+----------+-------------+----------------+-------------+------------+----------+-----------------+----------------+------------+--------------------+-------------------+----------------+------------+----------------+-------------+-------------------+----------------+
|  5433534|Ivo Ferreira|PESSOA DESENVOLVE...|TECNOLOGIA|     

In [ ]:
df_funcionarios_join_bonus.count()

1000

# Soma de Bonus por funcionário

In [29]:
df_func_soma_bonus = df_funcionarios_join_bonus \
                      .withColumn("soma_bonus",
                                  df_funcionarios_join_bonus["bonus_quadrante1"] +
                                  df_funcionarios_join_bonus["bonus_quadrante2"] +
                                  df_funcionarios_join_bonus["bonus_quadrante3"])

In [ ]:
df_func_soma_bonus.show(10)

+---------+--------------------+--------------------+--------------------+-------------+----------------+----------------+----------------+----------+
|matricula|                nome|               cargo|                area|salario_bruto|bonus_quadrante1|bonus_quadrante2|bonus_quadrante3|soma_bonus|
+---------+--------------------+--------------------+--------------------+-------------+----------------+----------------+----------------+----------+
|  5433534|        Ivo Ferreira|PESSOA DESENVOLVE...|          TECNOLOGIA|       3636.0|          1515.0|          1818.0|          2424.0|    5757.0|
|  5241166|Caetana da Magalhães|      JOVEM APRENDIZ|          TECNOLOGIA|      12726.0|          1515.0|          1212.0|          2424.0|    5151.0|
|  3027629|    Benjamim Cardoso|AUXILIAR ADMINIST...|       CONTABILIDADE|       3636.0|          1515.0|          1818.0|          1515.0|    4848.0|
|  2556888|     Diego da Campos|           ATENDENTE|RELACIONAMENTO CO...|       1515.0|      

In [30]:
df_soma_total_bonus_empresa = df_func_soma_bonus \
  .select(df_func_soma_bonus['soma_bonus']) \
  .groupBy() \
  .sum() \

In [31]:
df_soma_total_bonus_empresa.show()

+-----------------+
|  sum(soma_bonus)|
+-----------------+
|4942172.399999981|
+-----------------+



In [ ]:
df_soma_total_bonus_empresa.show()

+---------------+
|sum(soma_bonus)|
+---------------+
|      5429154.0|
+---------------+

